In [1]:
import sys
import os
sys.path.append(os.path.abspath('../src')) # include top level package in python path

In [2]:
from IPython.display import display, HTML
display(HTML("""
<style>
.container { 
    width: 100% !important;
}
</style>
"""))

In [3]:
import torch
from torch import nn, Tensor
from model.fuzzy_layer import FuzzyUnsignedConjunction, FuzzyUnsignedDisjunction, FuzzyNumKeepup, keepidx, fuzzy_dropup
from model.fuzzy_logic import ProductLogic, MinimumLogic, LukasiewiczLogic, DrasticLogic, SchweizerSklarLogic
from model.fuzzy_layer import FuzzyParam
from model.bool_logic import BoolLogic
from cache import TrainingRegime, TestMetric
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from util import shuffle

logic = BoolLogic()
flogic = ProductLogic()

In [4]:
device = 'cuda'

In [5]:
def true_conjunction(weights, xs):
    return logic.conjoin(logic.implies(weights, xs), dim=1)

In [6]:
from torch.utils.data import Dataset, DataLoader, RandomSampler

class WeightedDataset(Dataset):
    def __init__(self, N, D, K):
        super().__init__()
        self.samples = torch.zeros((N, D), dtype=torch.bool)
        self.samples[:,:K] = True
        self.samples = shuffle(self.samples, dim=-1)
        
    def __len__(self):
        return self.samples.size(0)
        
    def __getitem__(self, idx):
        return self.samples[idx]

class BinaryDataset(Dataset):
    def __init__(self, shape, N):
        self.samples = torch.distributions.bernoulli.Bernoulli(
            0.5
        ).sample(
            torch.Size([N, *shape])
        ).bool()
        
    def __len__(self):
        return self.samples.size(0)
    
    def __getitem__(self, idx):
        return self.samples[idx]

In [7]:
class CorrectParamMetric(TestMetric):
    def __init__(self, ideal_params):
        self.ideal_params = ideal_params
    
    def name(self):
        return "correct-param"

    def measure_model(self, model, _):
        params = (model.weights.value() > 0.5).flatten()
        no_correct = (self.ideal_params == params).sum()
        return (no_correct / params.numel()).item()
    
class ParamMeanDistanceMetric(TestMetric):
    def __init__(self, ideal_params):
        self.ideal_params = ideal_params
        
    def name(self):
        return "param-mean-distance"
    
    def measure_model(self, model, _):
        params = model.weights.value().flatten()
        crispness = (params - self.ideal_params.float()).abs().mean()
        return crispness.item()
    
class ParamMaxDistanceMetric(TestMetric):
    def __init__(self, ideal_params):
        self.ideal_params = ideal_params
        
    def name(self):
        return "param-max-distance"
    
    def measure_model(self, model, _):
        params = model.weights.value().flatten()
        crispness = (params - self.ideal_params.float()).abs().max()
        return crispness.item()
    
class CorrectOutputMetric(TestMetric):
    def __init__(self, ideal_params):
        self.ideal_params = ideal_params
        
    def name(self):
        return "correct-output"
    
    def measure_model(self, model, it):
        count = 0
        correct = 0
        for bs in it():
            bs = model._logic.encode(bs.to(device))
            pred = (model(bs) > 0.5).flatten()
            true = true_conjunction(self.ideal_params, bs)
            correct += (pred == true).sum().item()
            count += bs.size(dim=0)
        return (correct / count)
    
class OutputDistanceMetric(TestMetric):
    def __init__(self, ideal_params):
        self.ideal_params = ideal_params
        
    def name(self):
        return "output-distance"
    
    def measure_model(self, model, it):
        count = 0
        correct = 0
        for bs in it():
            bs = model._logic.encode(bs.to(device))
            pred = model(bs).flatten()
            true = true_conjunction(self.ideal_params, bs)
            correct += (pred - true.float()).abs().sum().item()
            count += bs.size(dim=0)
        return (correct / count)

In [8]:
def rand_n_true(shape, N):
    falses = torch.zeros(shape, dtype=torch.bool)
    falses = falses.view(-1)
    falses[:N] = True
    falses = falses[torch.randperm(falses.size(0))]
    return falses.view(*shape)

In [9]:
class ConjunctionRegime(TrainingRegime):
    def __init__(self, logic, no_dims=1000, no_terms=5, no_runs=1, keepn=None):
        super().__init__("./conjunctions/", no_runs)
        
        self.flogic = logic
        self.no_dims = no_dims
        self.no_terms = no_terms
        self.keepn = keepn
        
        self.weights = self.cache(
            "weights.pt", 
            lambda: rand_n_true([self.no_dims], self.no_terms).to(device)
        )
        
        self.tests = [
            OutputDistanceMetric(self.weights),
            CorrectOutputMetric(self.weights),
            CorrectParamMetric(self.weights),
            ParamMeanDistanceMetric(self.weights),
            ParamMaxDistanceMetric(self.weights),
        ]
        
        self.training_dataset = BinaryDataset([self.no_dims], 10_000)
        self.testing_dataset = BinaryDataset([self.no_dims], 10_000)
        
        self.optims = [None] * no_runs
        
    def get_optim(self, run_no):
        optim = self.optims[run_no - 1]
        if optim is None:
            model = self.get_loaded_model(run_no)
            optim = self.optims[run_no - 1] = (
                torch.optim.Adam(model.parameters(), lr=1e-2)
            )
        return optim
        
    def regime_str(self):
        keep_str = "" if self.keepn is None else " | KEEP N = %s" % self.keepn
        return (
            "CONJUNCTIONS | %s DIMENSIONS | %s TERMS%s"
            % (self.no_dims, self.no_terms, keep_str)
        ) 
        
    def regime_filename_elems(self):
        elems = [
            "conj", 
            "%sdims" % self.no_dims,
            "%sterms" % self.no_terms
        ]
        if self.keepn is not None:
            elems.append("keep%s" % self.keepn)
        return elems
        
    def training_dataloader(self, run_no):
        return DataLoader(self.training_dataset, batch_size=128)
    
    def testing_dataloader(self, run_no):
        return DataLoader(self.testing_dataset, batch_size=128)
    
    def training_step(self, run_no, model):
        optim = self.get_optim(run_no)
        
        def step(bs):
            bs = bs.cuda()
            cjs = true_conjunction(self.weights, bs)
            cj_hats = model(self.flogic.encode(bs)).squeeze()
            loss = (self.flogic.bin_xor(self.flogic.encode(cjs), cj_hats) ** 10).sum()
            
            optim.zero_grad()
            loss.backward()
            optim.step()
            
        return step
        
    def new_model(self):
        return FuzzyUnsignedConjunction(
            in_features=self.no_dims, 
            out_features=1, 
            logic=self.flogic, 
            keepn=30
        ).to(device)

In [ ]:
regime = ConjunctionRegime(logic=ProductLogic(), keepn=50, no_runs=10)
regime.load_latest_models()
regime.load_all_results()
for i in range(1, 10 + 1):
    regime.loop_until(i, 40)

[CONJUNCTIONS | 1000 DIMENSIONS | 5 TERMS | KEEP N = 50] Run #1, Epoch #1, Training:   0%|          | 0/79 [00…

[CONJUNCTIONS | 1000 DIMENSIONS | 5 TERMS | KEEP N = 50] Run #1, Epoch #1, Testing [output-distance]:   0%|   …

[CONJUNCTIONS | 1000 DIMENSIONS | 5 TERMS | KEEP N = 50] Run #1, Epoch #1, Testing [correct-output]:   0%|    …

[CONJUNCTIONS | 1000 DIMENSIONS | 5 TERMS | KEEP N = 50] Run #1, Epoch #2, Training:   0%|          | 0/79 [00…

[CONJUNCTIONS | 1000 DIMENSIONS | 5 TERMS | KEEP N = 50] Run #1, Epoch #2, Testing [output-distance]:   0%|   …

...

[CONJUNCTIONS | 1000 DIMENSIONS | 5 TERMS | KEEP N = 50] Run #6, Epoch #6, Testing [correct-output]:   0%|    …

[CONJUNCTIONS | 1000 DIMENSIONS | 5 TERMS | KEEP N = 50] Run #6, Epoch #7, Training:   0%|          | 0/79 [00…

[CONJUNCTIONS | 1000 DIMENSIONS | 5 TERMS | KEEP N = 50] Run #6, Epoch #7, Testing [output-distance]:   0%|   …

[CONJUNCTIONS | 1000 DIMENSIONS | 5 TERMS | KEEP N = 50] Run #6, Epoch #7, Testing [correct-output]:   0%|    …

[CONJUNCTIONS | 1000 DIMENSIONS | 5 TERMS | KEEP N = 50] Run #6, Epoch #8, Training:   0%|          | 0/79 [00…

In [ ]:
n_dims = 500

ds = BinaryDataset([n_dims], 10_000)
#ds = WeightedDataset(10_000, n_dims, n_dims - 10)
sampler = RandomSampler(ds, replacement=True, num_samples=300_000)
loader = DataLoader(ds, batch_size=64, sampler=sampler)

In [ ]:
regime.optims[0]

In [ ]:
weights = rand_n_true([n_dims], 30).cuda()

In [ ]:
def train_conj(model, optim):

    ret = {
        "losses": [],
        "diffs": [],
    }
    
    for bs in tqdm(loader):
        bs = bs.cuda()
        cjs = true_conjunction(weights, bs)
        cj_hats = model(flogic.encode(bs)).squeeze()
        #loss = ((cjs.float() - cj_hats).abs()).mean()
        loss = (flogic.bin_xor(flogic.encode(cjs), cj_hats) ** 10).sum()
        ret["losses"].append(loss.item())
        ret["diffs"].append(flogic.bin_xor(weights.float(), model.weights.value().flatten()))

        optim.zero_grad()
        loss.backward()
        optim.step()

    ret["losses"] = torch.Tensor(ret["losses"])
    ret["diffs"] = torch.cat([t.unsqueeze(0) for t in ret["diffs"]], dim=0)
        
    return ret

In [ ]:
cj_model = FuzzyUnsignedConjunction(in_features=n_dims, out_features=1, logic=flogic, keepn=30).cuda()
optim = torch.optim.Adam(cj_model.parameters(), lr=1e-2)

In [ ]:
#ret = train_conj(cj_model, optim)

In [ ]:
plt.plot(ret["losses"])

In [ ]:
(cj_model.weights.value() > 0.5).flatten()

In [ ]:
weights

In [ ]:
plt.plot(ret["diffs"].detach().numpy()[:], color="black", alpha=0.03)

In [ ]:
ret["diffs"].shape

In [ ]:
print(ds)

In [ ]:
ds.samples.shape

In [ ]:
xs = torch.rand((2,2))
idx = torch.Tensor([[0,1],[1,0]]).bool()
xs[idx] = (xs[idx] > 0.5).float()
xs

In [ ]:
len(torch.ones((4,5)).shape)

In [ ]:
xs = torch.arange(3) + torch.zeros(2,1)
idxs = keepidx((2,3), n=2, dim=1)
vals = xs[idxs]
vals = vals.reshape(*xs.shape[:-1], 2)
vals

In [ ]:
take_rand_n(torch.arange(25).view(5,5), 3, dim=0)

In [ ]:
torch.arange(25).view(5,5)

In [ ]:
for i in range(0, 1):
    plt.plot(
        regime.results[i]['epoch'], 
        regime.results[i]['correct-output'],
        color="black",
        alpha=0.1
    )